# Interactive TinyMPC Example in Python (extended version)
We demonstrate an interactive workflow in Python with TinyMPC where you can generate C code and interact with it via Python. This example shows the entire workflow from cartpole nonlinear dynamics

Restart kernel if something breaks.

Load necessary packages, make sure to install `tinympc` ([README.md](../README.md))

In [1]:
import tinympc
import numpy as np
import math
np.set_printoptions(precision=3, suppress=True)

Define the class and compile original TinyMPC code to get a generic shared/dynamic library

**PLEASE CHANGE `tinympc_python_dir` TO YOUR ABSOLUTE PATH**

In [2]:
prob = tinympc.TinyMPC()

tinympc_python_dir = "/home/khai/SSD/Code/tinympc-python"  # Your absolute path to the tinympc-python directory, you may only need to change this

tinympc_dir = tinympc_python_dir + "/tinympc/TinyMPC"  # Path to the TinyMPC directory (C code)
prob.compile_lib(tinympc_dir)  # Compile the library

True

Load the generic shared/dynamic library 

In [3]:
lib_dir = tinympc_dir + "/build/src/tinympc/libtinympcShared.so"
prob.load_lib(lib_dir)  # Load the library

We start by building the cartpole nonlinear dynamics and linearize around the upright position. Try LQR to see if it work!

In [4]:
import autograd.numpy as np
from autograd import grad, jacobian

freq = 100  # frequency of the controller and dynamics Hz
dt = 1/freq  # time step

# cartpole dynamics with theta = 0 at the down position
def cartpole_dynamics(x, u):
    g = -9.8  # gravity m/s^2
    m = 0.2  # mass of the pole kg
    M = 0.5  # mass of the cart kg
    l = 0.3  # length of the pole m
    
    x, x_dot, theta, theta_dot = x
    u = u[0]

    x_ddot = (u + m * l * theta_dot**2 * np.sin(theta) - m * g * np.sin(theta) * np.cos(theta)) / (M + m * np.sin(theta)**2)
    theta_ddot = (-u * np.cos(theta) - m * l * theta_dot**2 * np.sin(theta) * np.cos(theta) + (M + m) * g * np.sin(theta)) / (l * (M + m * np.sin(theta)**2))
    
    return np.array([x_dot, x_ddot, theta_dot, theta_ddot])

def rk4(x, u):
    f = cartpole_dynamics
    k1 = f(x, u)
    k2 = f(x + dt * k1 / 2, u)
    k3 = f(x + dt * k2 / 2, u)
    k4 = f(x + dt * k3, u)
    return x + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

def cartpole_dynamics_rk4(x, u):
    return rk4(x, u)

# Linearize the dynamics around x0, u0
A_jac = jacobian(cartpole_dynamics_rk4, 0)  # jacobian wrt x
B_jac = jacobian(cartpole_dynamics_rk4, 1)  # jacobian wrt u

x0 = np.array([0, 0, math.pi, 0.0]) 
u0 = np.array([0.])
Anp = A_jac(x0, u0)  # jacobian of the dynamics wrt x at x0, u0
Bnp = B_jac(x0, u0)  # jacobian of the dynamics wrt u at x0, u0
print("A = \n", Anp)
print("B = \n", Bnp)

x_all = []
# simulate the dynamics with the zero controller
# x = np.array([0, 0.1, 0, 0])
# for i in range(300):
#     x = cartpole_dynamics_rk4(x, u0)
#     x_all.append(x)

# Riccati recursion on the linearized dynamics
Q = np.diag([1, 1, 1, 1])
R = np.diag([1])
P = Q
for i in range(100):
    P = Q + Anp.T @ P @ Anp - Anp.T @ P @ Bnp @ np.linalg.inv(R + Bnp.T @ P @ Bnp) @ Bnp.T @ P @ Anp
    # print(P)
    K = np.linalg.inv(R + Bnp.T @ P @ Bnp) @ Bnp.T @ P @ Anp
# print("K = \n", K)

# LQR controller
def lqr_controller(x):
    return -K @ x

x = np.array([0, 0, math.pi - 0.1, 0.0])  # initial state
NSIM = 300
# simulate the dynamics with the LQR controller
for i in range(NSIM):
    u = lqr_controller(x - x0)
    x = cartpole_dynamics_rk4(x, u)
    x_all.append(x)

# Go to the visualization part and run it to see the trajectory `x_all`


A = 
 [[1.    0.01  0.    0.   ]
 [0.    1.    0.039 0.   ]
 [0.    0.    1.002 0.01 ]
 [0.    0.    0.458 1.002]]
B = 
 [[0.   ]
 [0.02 ]
 [0.   ]
 [0.067]]


Here we setup problem data and settings for TinyMPC

In [5]:
n = 4
m = 1
N = 10

A = Anp.transpose().reshape((n * n)).tolist()
B = Bnp.reshape((n * m)).tolist()
print(A)
print(B)
Q = [10.0, 1, 10, 1]
R = [1.0]
rho = 0.1

x_min = [-5.0] * n * N  # state constraints
x_max = [5.] * n * N  # state constraints
u_min = [-5.] * m * (N - 1)  # force constraints
u_max = [5.] * m * (N - 1)  # force constraints

abs_pri_tol = 1.0e-3  # absolute primal tolerance
abs_dual_tol = 1.0e-3  # absolute dual tolerance
max_iter = 100  # maximum number of iterations
check_termination = 1  # whether to check termination and period

# Setup problem data
prob.setup(n, m, N, A, B, Q, R, x_min, x_max, u_min, u_max, rho, abs_pri_tol, abs_dual_tol, max_iter, check_termination)

[1.0, 0.0, 0.0, 0.0, 0.01, 1.0, 0.0, 0.0, 0.0001960746977777778, 0.039229879111111116, 1.002287538140741, 0.457681922962963, 6.533333333333335e-07, 0.00019607469777777783, 0.010007622222222222, 1.002287538140741]
[0.00010001088888888889, 0.020004355555555558, 0.0003334603703703704, 0.06671748148148149]


True

After define the problem, we generate the tailored code with above data. 

Here we compile it for interactive Python script but you can use it directly for your applications/systems

In [6]:
output_dir = tinympc_python_dir + "/generated_code1"  # Path to the generated code
prob.tiny_codegen(tinympc_dir, output_dir)
prob.compile_lib(output_dir)

True

Load the compiled shared/dynamic library

In [7]:
prob.load_lib(output_dir + "/build/tinympc/libtinympcShared.so")  

Run the interactive MPC example which calls the generated code, use real nonlinear dynamics

This works with pointers then C data is modified in each kernel session. If you want to run from the original setup, you may need to change data back or 
restart kernel.

In [8]:
x = [0.5, 0, -0.4 + math.pi, 0.1] # Initial state
u = [0.0] * m * (N - 1)  # List of control inputs in horizon
x_all = []  # List of all stored states
delta_x_noise = (x - x0) * 1
x_get = [0.0] * n * (N)  # List of control inputs in horizon

# Use delta because MPC uses the linearized dynamics around upright position
# delta_xref_next = [1.0, 0.0, 0.0, 0.0] * N  # List of reference states in horizon, col-major

# Set the reference state to 0 as well as reset
delta_xref = np.repeat(np.array([[0.0], [0.0], [0.0], [0.0]]), N, 1).transpose().reshape(n*N).tolist()
prob.set_xref(delta_xref, 1)  # Set the reference state to C code

# Set the reference x to 1 at step 200
delta_xref_next = np.repeat(np.array([[1.0], [0.0], [0.0], [0.0]]), N, 1).transpose().reshape(n*N).tolist()

print("=== START INTERACTIVE MPC ===")

NSIM = 400
for i in range(NSIM):
    # 1. Set initial state from measurement    
    prob.set_x0(delta_x_noise, 0)  # Set initial state to C code
    
    # 2. Set the reference state if needed
    # At step 200, set x = 1
    if (i == 200):
        prob.set_xref(delta_xref_next, 0)  # Set the reference state to C code

    # 3. Solve the problem
    prob.solve(0)  # Call the solve in C code

    # 4. Get the control input
    prob.get_u(u, 0)  # Get the control input from C code

    # 5. Simulate the dynamics
    x = cartpole_dynamics_rk4(np.array(x).reshape((n, 1)), u)

    noise = np.random.normal(0, 0.01, (n, 1))
    delta_x_noise = (x + noise.reshape((n, 1)) - x0.reshape((n,1))).reshape(n).tolist()
    x = x.reshape(n).tolist()
    x_all.append(x)

print(len(x_all))

=== START INTERACTIVE MPC ===
[[0.5004332956667601, 0.08667649372918218, 2.743287837312194, 0.23920684123838362], [0.501706794773503, 0.16805272993963039, 2.7462978845040857, 0.3630967925172838], [0.5037909159223268, 0.2488127670518549, 2.750544111852759, 0.48656237823436216], [0.5066818784622372, 0.32943304048339994, 2.7560320800038998, 0.6115654474046762], [0.5103824713772751, 0.4107516430640932, 2.7627926029475973, 0.7411991912208304], [0.5148998039252564, 0.492794391256265, 2.7708750103003426, 0.8760735456185857], [0.520242310886574, 0.5758007999358771, 2.7803381554618496, 1.0174849830970079], [0.526416888924551, 0.6592230269194567, 2.7912417311946216, 1.1643011082335075], [0.5334295362982205, 0.7434299067721394, 2.803648463352023, 1.318264783506473], [0.5412886187569139, 0.8285258861959583, 2.817634528644238, 1.4803228180895713], [0.5500029989272333, 0.9145056319116522, 2.8332839834058854, 1.6511033875240857], [0.5595820049685774, 1.0014674606534182, 2.85068893681256, 1.8315887816

Visualize the robot and trajectory

In [9]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Set up the figure and axis for plotting
fig, ax = plt.subplots()
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-1, 1)

# Initialize the cartpole visualization
cart, = ax.plot([], [], 'bo', markersize=20)
pole, = ax.plot([], [], 'r-', linewidth=4)

def init():
    cart.set_data([], [])
    pole.set_data([], [])
    return cart, pole

def update(frame):
    x = x_all[frame]
    # Update the cart position
    cart.set_data([x[0]], [0])

    # Update the pole position, zero angle points down since we use nonlinear dynamics
    pole.set_data([x[0], x[0] + 0.5*math.sin(x[2])], [0, -0.5 * math.cos(x[2])])
    # print(frame)
    # if frame==NSIM-1:
    #     ani.event_source.stop()  # Stop the animation if the episode is 
    return cart, pole

# Create the animation
ani = FuncAnimation(fig, update, frames=NSIM, init_func=init, blit=False, interval=10)

# Display the animation (may not display anything in notebook :D, that's why I save it to a file)
plt.show(ani)

<IPython.core.display.Javascript object>

Save the animation

In [10]:
ani.save('cartpole2.mp4', writer='ffmpeg', fps=30, dpi=200)

After testing MPC procedure with the generated code, you need to deploy it for your applications/systems. Stay tuned for Teensy and STM32 deployment tutorials. 

Your `tiny_main` may look like this

```C
int main()
{
    int exitflag = 1;
    TinyWorkspace* work = tiny_data_solver.work;
    tiny_data_solver.work->Xref = tiny_MatrixNxNh::Zero();
    tiny_data_solver.work->Uref = tiny_MatrixNuNhm1::Zero();
    tiny_data_solver.settings->max_iter = 150;
    tiny_data_solver.settings->en_input_bound = 1;
    tiny_data_solver.settings->en_state_bound = 1;

    tiny_VectorNx x0, x1; // current and next simulation states
    x0 << 0.0, 0, 0.1, 0; // initial state

    int i = 0;
    for (int k = 0; k < 300; ++k)
    {
        printf("tracking error at step %2d: %.4f\n", k, (x0 - work->Xref.col(1)).norm());

        // 1. Update measurement
        work->x.col(0) = x0;

        // 2. Update reference (if needed)
        // you can also use C wrapper (intended for high-level languages) 
        // by including tiny_wrapper.hpp and call `set_xref(...)` function

        // 3. Reset dual variables (if needed)
        work->y = tiny_MatrixNuNhm1::Zero();
        work->g = tiny_MatrixNxNh::Zero();

        // 4. Solve MPC problem
        exitflag = tiny_solve(&tiny_data_solver);

        // if (exitflag == 0)
        // 	printf("HOORAY! Solved with no error!\n");
        // else
        // 	printf("OOPS! Something went wrong!\n");
        // 	// break;

        std::cout << work->iter << std::endl;
        std::cout << work->u.col(0).transpose().format(CleanFmt) << std::endl;

        // 5. Simulate forward
        // work->u.col(0) = -tiny_data_solver.cache->Kinf * (x0 - work->Xref.col(0));
        x1 = work->Adyn * x0 + work->Bdyn * work->u.col(0);
        x0 = x1;
        // std::cout << x0.transpose().format(CleanFmt) << std::endl;
    }
}
```